<a href="https://colab.research.google.com/github/Sohdjeukeu/Labo_GTI_720/blob/main/labo_1_01_G.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Description
Le but de ce laboratoire est de vous familiariser aux enjeux de vie privée des données dépersonalisées et aggrégés ainsi qu'aux propriétés de k-anonymat et de l-diversité. On utilisera ici une version modifiée (contenant des `DCP`) du jeu de données [UCI Adult Income](https://archive.ics.uci.edu/ml/datasets/adult) contenant des informations démographiques du recensement américain de 1994. Ce laboratoire est réalisé avec Python et les librairies `pandas`, `numpy` et `matplotlib`.

In [1]:
%pip install pyspark

In [ ]:
%pip --version # version du gestionnaire de package utlisée

pip 24.1.2 from /usr/local/lib/python3.11/dist-packages/pip (python 3.11)


## Partie 0: Installation de l'environement de travail (5pts)

**Question 0** [5 pts]:
- Installer `uv` en suivant cette [documentation](https://docs.astral.sh/uv/)
- Utiliser `uv` pour installer `Python 3.13`
- Installer les librairies suivantes dans votre environement virtuel local
    - `pandas`
    - `numpy`
    - `matplotlib`

# importation des librairies

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


## Partie 1: Attaques ciblant un jeu de données dépersonalisées (31 pts)
Un renseignement personnel est `dépersonnalisé` si ce dernier ne permet plus d’identifier *directement* la personne concernée. La dépersonilisation consiste donc à retirer les identifiants direct du jeu de données.

In [3]:
# importation du jeux de données
adult = pd.read_csv('https://github.com/tisl-lab/data-privacy-class/blob/main/data/adult_with_pii.csv?raw=true')

In [4]:
# affichache des 5 premières lignes du jeux de données
adult.head(5)

,Name,DOB,SSN,Zip,Age,Workclass,fnlwgt,Education,Education-Num,Marital Status,Occupation,Relationship,Race,Sex,Capital Gain,Capital Loss,Hours per week,Country,Target
0,Karrie Trusslove,9/7/1967,732-14-6110,64152,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,Brandise Tripony,6/7/1988,150-19-2766,61523,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,Brenn McNeely,8/6/1991,725-59-9860,95668,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,Dorry Poter,4/6/2009,659-57-4974,25503,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,Dick Honnan,9/16/1951,220-93-3811,75387,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


**Question 1** [3 pts]: Identifier les `identifiants directs` de ce jeu de données. Justifier pourquoi vous avez choisis ces attributs.
Conseils: Référer vous à la définition de donnée à caractère personnel définie dans le cours 1.

In [5]:
if isinstance(adult, pd.DataFrame):
    print("C'est un DataFrame")
else:
    print("Ce n'est pas un DataFrame")

C'est un DataFrame


# Vérifier les doublons pour chaque colonne

In [7]:
df = adult

colonne_sans_doublon = []
colonne_sans_doublon.clear()

for colonne in df.columns:
  if not df[colonne].duplicated(keep=False).any():
    colonne_sans_doublon.append(colonne)
colonne_sans_doublon

['Name']

D'après le code ci-dessus, on voit qu'il n'y a que la métadonnée Name qui est unique et représente le seul identifiant direct: ## Name




**Question 2** [5 pts]: A l'aide de ces `identifiants directs`, construire la version dépersonalisée de ce jeu de données.

Elle va consister à réduire le risque d'identifier directement une personne et dansce contexte précis, la metadonnée Name a des valeurs unique et il permet d'identifier directement une personne.Nous allons donc supprimer la metadonée Name pour depersonnaliser le jeu de données adult.

In [8]:
#df_copy.drop(labels='Name', axis=1)

In [9]:
def depersonaliser_adult():
    #VOTRE CODE ICI
    df_A_depersonnaliser = adult.copy()
    df_A_depersonnaliser.drop(labels='Name', axis=1,inplace=True)
    return df_A_depersonnaliser

In [11]:
# Appel de la methode depersonaliser_adult() pour afficher la dataset adult sans son identifiant direct Name
depersonaliser_adult()


,DOB,SSN,Zip,Age,Workclass,fnlwgt,Education,Education-Num,Marital Status,Occupation,Relationship,Race,Sex,Capital Gain,Capital Loss,Hours per week,Country,Target
0,9/7/1967,732-14-6110,64152,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,6/7/1988,150-19-2766,61523,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,8/6/1991,725-59-9860,95668,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,4/6/2009,659-57-4974,25503,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,9/16/1951,220-93-3811,75387,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,10/29/1961,212-61-8338,41328,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,6/28/1952,737-32-2919,94735,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,8/9/1966,314-48-0219,49628,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,5/19/1978,647-75-3550,8213,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


**Question 3** [1 pts]: Afficher les 5 premières ligne du jeu de données dépersonalisée

In [14]:
depersonaliser_adult().head() # affiche les 5 premiers lignes

,DOB,SSN,Zip,Age,Workclass,fnlwgt,Education,Education-Num,Marital Status,Occupation,Relationship,Race,Sex,Capital Gain,Capital Loss,Hours per week,Country,Target
0,9/7/1967,732-14-6110,64152,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,6/7/1988,150-19-2766,61523,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,8/6/1991,725-59-9860,95668,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,4/6/2009,659-57-4974,25503,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,9/16/1951,220-93-3811,75387,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


**Question 4** [3 pts]: En utilisant `pandas`, écrire un code qui renvoit uniquement la ligne corespondant à l'enregistrement de `Faith McCloughlin` à partir du jeu de donnée original `adult`

In [ ]:
df['Name']


,Name
0,Karrie Trusslove
1,Brandise Tripony
2,Brenn McNeely
3,Dorry Poter
4,Dick Honnan
...,...
32556,Ardyce Golby
32557,Jean O'Connor
32558,Reuben Skrzynski
32559,Caye Biddle


In [36]:
def get_McCloughlin_row():
    # VOTRE CODE ICI:
    # définissons les critères de recherche, il sera basé uniquemnt sur le nom
    critere_reherche = {
        'Name':'Faith McCloughlin'
    }
    # filtrer le dataset adult pour trouver la ligne correspondante
    for index, valeur in adult['Name'].items():
      if valeur == critere_reherche['Name']:
        resultat = adult.iloc[index]
        return resultat
    return None

In [37]:
get_McCloughlin_row()

,11019
Name,Faith McCloughlin
DOB,12/25/1984
SSN,718-79-8158
Zip,41277
Age,30
Workclass,Private
fnlwgt,132565
Education,Some-college
Education-Num,10
Marital Status,Never-married


In [33]:
McCloughlin_row = get_McCloughlin_row()

In [35]:
#assert len(McCloughlin_row) == 1
#assert McCloughlin_row['SSN'].iloc[0] == '718-79-8158'

**Question 5** [2 pts]:  À l'aide de l'enregistrement `McCloughlin_row`, récuper les quasi-identifiants `Zip`, `Age` et `Sex` de `Faith McCloughlin`.

In [41]:
liste_Zip_Age_Sex = McCloughlin_row[['Zip','Age','Sex']].tolist()
#liste_Zip_Age_Sex

print(f'son Zip:{liste_Zip_Age_Sex[0]}; son age:{liste_Zip_Age_Sex[1]}; son sex: {liste_Zip_Age_Sex[2]}')

son Zip:41277; son age:30; son sex: Male


**Question 6** [5 pts]:  En utilisant ces quasi-identifiants, effectuer une attaque de `ré-identification` de `Faith McCloughlin` à partir du jeu de données dépersonalisé `Faith McCloughlin`.

L'attaque de ré-identification est une enace significative pour la protection des données personnelles) va consister à reconstruire l'identité des individus du jeu de données  à  partir des données qui étaient supposées anonymiser.

In [53]:
jeuDonneeesDepersonnaliser = depersonaliser_adult()
#jeuDonneeesDepersonnaliser


In [54]:
def recover_McCloughlin_row():
    # VOTRE CODE ICI
    # chargeons le jeu de données depersonnaliser
    JeuDonneeesDepersonnaliser = depersonaliser_adult()
    # Filtrer les enregistrements corresponds simultanément aux quasi-identifiants de Faith McCloughlin
    Zip_faith = 41277
    age_faith = 30
    sex_faith = 'Male'
    # filtrer le jeu de données depersonnaliser pour trouver la ligne correspondante à Faith McCloughlin
    for index, valeur in JeuDonneeesDepersonnaliser.iterrows():
      if(valeur['Zip']==Zip_faith and valeur['Age']==age_faith and valeur['Sex']==sex_faith):
        return valeur
    return None


In [55]:
recover_McCloughlin_row()

,11019
DOB,12/25/1984
SSN,718-79-8158
Zip,41277
Age,30
Workclass,Private
fnlwgt,132565
Education,Some-college
Education-Num,10
Marital Status,Never-married
Occupation,Craft-repair


In [42]:
McCloughlin_recovered_row = recover_McCloughlin_row()

**Question 7** [2 pts]:  Montrer que la connaissance du `Zip` suffit pour ré-identifier `Faith McCloughlin`

In [64]:
# filtrons le jeu de données depersonnaliser uniquement sur le zip de Faith McCloughlin pour voir si nous le retrouvons
# chargeons le jeu de données depersonnaliser
JeuDonneeesDepersonnaliser = depersonaliser_adult()
 # Filtrer les enregistrements uniquement sur quasi-identifiants  Zip de Faith McCloughlin et verifions s'il retourne uniquement Faith McCloughlin
merlin = JeuDonneeesDepersonnaliser['Zip']
JeuDonneeesDepersonnaliser.dtypes # permet de connaitre le type de chaque colonne
merlin.dtype


dtype('int64')

In [76]:
# filtrons le jeu de données depersonnaliser uniquement sur le zip de Faith McCloughlin pour voir si nous le retrouvons

# chargeons le jeu de données depersonnaliser
JeuDonneeesDepersonnaliser = depersonaliser_adult()


  # définissons les critères de recherche, il sera basé uniquemnt sur le zip de Faith McCloughlin

critere_reherche = {
        'Zip':41277
    }
# Filtrer les enregistrements uniquement sur quasi-identifiants  Zip de Faith McCloughlin et verifions s'il retourne uniquement Faith McCloughlin

Taille = JeuDonneeesDepersonnaliser.shape  # retourne le nombre de ligne et de colonne
type(Taille)



tuple

**Question 8** [4 pts]:  Peut-on dire la même chose de l'attribut `Age`? Même question pour l'attribut `Sex`.

**Question 9** [2 pts]:  Combien de personnes peut-on réidentifier de manière unique avec la connaissance de l'attribut `Zip`? Même question pour l'attribut `DOB`.

**Question 10** [4 pts]:  Combien de personnes de personnes peut-on réidentifier avec une chance de 1/2 (resp. 1/3) avec la connaissance de l'attribut `Zip`? Même question pour l'attribut `DOB`

## Partie 2: Attaques ciblant un jeu de données aggrégées (25 pts)
La partie 1 montre que les jeux de données dépersonnalisés peuvent être problématique car ils permettent avec très peu d'information de réidentifier des personnes concernées.
Dans cette partie, on s'intéresse à ce que les données aggrégées peuvent révéler sur les individus contenu dans les jeux de données originaux.

### Problème des petits groupes

**Question 11** [2 pts]: À l'aide de la fonction `mean`, calculer l'âge moyen des individus contenus dans le jeu de données `adult`.

**Question 12** [2 pts]: À l'aide des fonctions `mean` et `groupby`, calculer l'âge moyen *groupé par pays* des individus contenus dans le jeu de données `adult`.

**Question 13** [2 pts]: À l'aide de la fonction `value_counts`, montrer les statistiques du nombre de personnes par pays du jeu de données `adult`.

**Question 14** [5 pts]:  Que peut-on déduire de l'âge d'une certaine personne originaire des Pays-bas ?

### Attaques de différenciation

Supposons maintenant que les données agrégées sont publiées chaque années et que le jeu de donnée `adult` contient les enregistrements de l'année courrante.

**Question 15** [2 pts]: À l'aide de la fonction `sum` calculer la somme des heures de travail (`Hours per week`) des individus contenus dans le jeu de données `adult` pour l'année courante.

**Question 16** [2 pts]: Sachant que seulement `Faith McCloughlin` a été ajouté au jeu de données cette année, calculer la somme des heures de travail des individus contenus dans le jeu de données `adult` l'année précédente.

**Question 17** [5 pts]: Déduire le nombre d'heures de travail de `Faith McCloughlin`

**Question 18** [5 pts]: Proposer une méthode pour déduire le nombre d'heures de travail si au lieu de la `somme des heures de travail`, c'est la `moyenne des heures de travail` qui est publiée chaque année.

## Partie 3: Anonymat (39 pts)


Pour cette partie on utilisera le jeu de donnée suivant:

In [ ]:
# importation du jeux de données
adult_df = pd.read_csv('https://github.com/tisl-lab/data-privacy-class/blob/main/data/adult_small.csv?raw=true')

**Question 19** [3 pts]: Expliquer en trois ou quatre phrases un algorithme pouvant vérifier étant donné un jeu de données `D`, une liste `QIDs` de quasi-identifiants et un paramètre `k` que `D` est **k-anonymisé**

**Question 20** [5 pts]: Proposer une implémentation de cet algorithme qui devra prendre en entrée un dataframe `df`, une liste de quasi-identifiant `QIDs` et le paramètre de k-anonymat `k`.

**Question 21** [2 pts]: Est-ce que ce le jeu de données `adult_df` satisfait le 2-anonymat pour QIDs = `["Sex", "Race"]`?

**Question 22** [3 pts]: Donner la liste des pairs de quasi-identifiants pour lesquels `adult_df` satisfait le 2-anonymat.

**Question 23** [3 pts]: Donner la liste des pairs de quasi-identifiants pour lesquels `adult_df` satisfait le 5-anonymat.

**Question 24** [3 pts]: Donner la liste des triplets de quasi-identifiants pour lesquels `adult_df` satisfait le 2-anonymat.

**Question 25** [3 pts]: Expliquer en trois ou quatre phrases comment l'algorithme précédent pourrait être modifié pour vérifier qu'un jeu de données anonymisé satisfait la **l-diversité** pour un attribut sensible `s`.

**Question 26** [5 pts]: Proposer une implémentation de cet algorithme qui devra prendre en entrée un dataframe `df`, une liste de quasi-identifiant `QIDs`, un attribut sensible `s`, le paramètre de k-anonymat `k` et le paramètre de l-diversité `l`.

**Question 27** [2 pts]:  Est-ce que ce le jeu de données `adult_df` satisfait au 3-anonymat pour QIDs = `["Sex", "Race"]` et à la 2-diversité pour l'attribut `Occupation`?

**Question 28** [5 pts]:  Étant donné le jeu de données `adult_df` qui satisfait le 3-anonymat pour QIDs = `["Sex", "Race"]`, parmi les attributs `["Age", "Workclass", "Relationship", "Country"]`, lesquels pourraient permettre une attaque d'homogénéité s'ils étaient publiés comme attributs sensibles?

**Question 29** [5 pts]: Proposer un algorithme pour généraliser l'attribut `Zip` en transformant les `n` derniers chiffres en `0`. Votre algorithme devra retourner la version transformée du jeu de données.